In [ ]:
#Logistic Regression Modeling
#Building an xplainable scorecard using a simple logistic regression model while ensuring interpretability,compliance and stability.

In [2]:
import pandas as pd
import numpy as np
import matplotlib as pyplot
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

df= pd.read_csv("credit_default.csv")

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

features = ['credit_limit_capped', 'AGE', 'delinquent_sept']
X = df[features]
y = df['default.payment.next.month']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

coeff_df = pd.DataFrame({
    'feature': features,
    'coefficient': model.coef_[0]
})

coeff_df


,feature,coefficient
0,credit_limit_capped,-6.902437e-06
1,AGE,-3.675482e-09
2,delinquent_sept,7.476935e-11


Magnitudes extremely off.Added scaling

In [ ]:
#Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

coeff_df = pd.DataFrame({
    'feature': features,
    'coefficient': model.coef_[0]
})

coeff_df

,feature,coefficient
0,credit_limit_capped,-0.334596
1,AGE,0.094840
2,delinquent_sept,0.741095


Deliquent_sept is strong and positive meaning if borrower was deliquent then PD increases significantly.
Negative coefficient of credit_limit shows higher credit limit=llower PD
Age ..older age increases PD.

07.Model Performance

In [26]:
from sklearn.metrics import roc_auc_score, classification_report

# Predictions
y_pred_proba = model.predict_proba(X_test)[:, 1]

# AUC
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score:", auc)

# Classification report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


AUC Score: 0.724830668773191
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      7040
           1       0.54      0.36      0.43      1960

    accuracy                           0.79      9000
   macro avg       0.69      0.64      0.65      9000
weighted avg       0.77      0.79      0.78      9000



AUC above 0.65 indicates good discriminatory power and can distinguish defaulters from non-defaulters.


In [27]:
#Convert AUC to Gini
gini = 2 * auc - 1
print("Gini:", gini)


Gini: 0.44966133754638205


In [28]:
#KS Statistic 
import numpy as np
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
ks = max(tpr - fpr)

print("KS Statistic:", ks)


KS Statistic: 0.3677208951762523
